<a href="https://colab.research.google.com/github/AmoreHH/data-course-sample/blob/main/W2/W2_content_based_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import gzip, json
import warnings
warnings.filterwarnings('ignore')

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
# !wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
# !wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

In [ ]:
metadata = getDF('meta_All_Beauty.json.gz')
ratings = pd.read_csv('All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [ ]:
# 刪除重複欄位 
a = metadata.astype(str).drop_duplicates(keep=False)
print(metadata.shape)
print(a.shape)

(32892, 19)
(32084, 19)


In [ ]:
# 將空白 或 空集合[] 轉為 NaN，計算na有多少
b =a.replace(to_replace='', value=np.nan)
d = b.mask(b.applymap(str).eq('[]'))
NAN = [(c, d[c].isna().mean()*100) for c in d]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
print(NAN.sort_values(by='percentage', ascending=False))

        column_name  percentage
0          category  100.000000
3               fit  100.000000
6             tech2  100.000000
1             tech1   99.968832
14             date   99.940780
8           feature   99.198978
13     similar_item   95.960603
5          also_buy   79.687695
10        also_view   75.464406
15            price   64.976312
2       description   54.665877
17         imageURL   49.691435
18  imageURLHighRes   49.691435
7             brand   48.207829
9              rank    0.938162
4             title    0.003117
12         main_cat    0.000000
11          details    0.000000
16             asin    0.000000


In [ ]:
too_much_empty = NAN[NAN['percentage']>90]
too_much_empty=too_much_empty['column_name'].tolist()
too_much_empty

['category', 'tech1', 'fit', 'tech2', 'feature', 'similar_item', 'date']

In [ ]:
df = d.drop(too_much_empty, axis=1)
df.shape

(32084, 12)

In [ ]:
# 參考這篇把price 轉成數值 https://www.kaggle.com/rastoginisha97/assignment-05
df.price=df.price.str.replace("$","")
#One of the price cells, also included text, so that needs to be modified too
def priceonly(x):
    try:
        x=float(x)
        return x
    except ValueError:
        return None

df.price=df.price.apply(lambda x: priceonly(x))


In [ ]:
# 接下來是處理 期初測試時也有做的  rank，分成排名(數值)跟所屬類別
df['rank_n'] = df['rank'].str.split(expand = True)[0]
df['rank_n'] = df['rank_n'].str.replace(',','')
df['rank_n'] = pd.to_numeric(df['rank_n'], errors='coerce')

In [ ]:
df['rank_c'] = df['rank'].str.split('in', expand = True, n=2) [1]
df['rank_c']=df['rank_c'].str.replace('&amp;', '&') 
df['rank_c'] = df['rank_c'].str.split('(', expand = True) [0]
print(df['rank_c'].value_counts())

 Beauty & Personal Care       31712
 Grocery & Gourmet Food          37
 Health & Household              21
 Sports & Outdoors                3
 Toys & Games                     3
 Baby                             2
 Cloth                            2
 Home & Kitchen                   1
 Automotive                       1
 Tools & Home Improvement         1
Name: rank_c, dtype: int64


In [ ]:
df = df[df.rank_c == " Beauty & Personal Care "]
df.shape

(31712, 14)

In [ ]:
metadata = df.copy()

## 整理 description的資料內容

In [ ]:
a = metadata.description.dropna()
print(a.head())
print('\n')
print(a[2])

0    ["Loud 'N Clear Personal Sound Amplifier allow...
1    ['No7 Lift & Luminate Triple Action Serum 50ml...
2    ["No7 Stay Perfect Foundation now stays perfec...
4    ['Lacto Calamine Skin Balance Daily Nourishing...
5    ['Mary Kay Satin Hands Peach Hand Cream Travel...
Name: description, dtype: object


["No7 Stay Perfect Foundation now stays perfect for longer! Up to 24 hours wear for non-stop complexion perfection. Life-proof foundation for a naturally flawless complexion with super-staying power, uniquely matched to your skin tone using No7 Stay Perfect. This comfortable foundation feels lightweight and breathable, and gives an even-toned, beautifully smooth complexion that looks and feels fresh all day long. It can be relied upon to stay perfect; weather-proof and transfer-proof, it won't cake or flake on dry skin, nor slide or shine on oily skin. Up to 24 hours wear for non stop complexion perfection!"]


In [ ]:
metadata["description_n"] = metadata["description"].dropna().apply(lambda x : eval(x))
a = metadata["description_n"].apply(pd.Series)

In [ ]:
metadata['description'] = a[0]

In [ ]:
#沒有 description的話，用 title補進去，就可以一起做文字項分析 
metadata.dropna(subset=['title'], inplace=True) #dropping records where tiltle is missing. 
metadata['description'].fillna(metadata['title'], inplace=True) #imputing missing discription with titles.

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'&amp;', '', text)
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    return text

In [ ]:
# 整理 description 的內容 把特殊符號這些去掉 
import string
import re
metadata['description'] = metadata['description'].apply(clean_text)

In [ ]:
metadata.head()

,description,title,also_buy,brand,rank,also_view,details,main_cat,price,asin,imageURL,imageURLHighRes,rank_n,rank_c,description_n
0,loud n clear personal sound amplifier allows y...,Loud 'N Clear&trade; Personal Sound Amplifier,NaN,idea village,"2,938,573 in Beauty & Personal Care (",NaN,{'ASIN: ': '6546546450'},All Beauty,NaN,6546546450,NaN,NaN,2938573.0,Beauty & Personal Care,[Loud 'N Clear Personal Sound Amplifier allows...
1,lift luminate triple action serum by boots,No7 Lift &amp; Luminate Triple Action Serum 50...,"['B01E7LCSL6', 'B008X5RVME']",NaN,"872,854 in Beauty & Personal Care (",NaN,"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,44.99,7178680776,NaN,NaN,872854.0,Beauty & Personal Care,[No7 Lift & Luminate Triple Action Serum 50ml ...
2,stay perfect foundation now stays perfect for...,No7 Stay Perfect Foundation Cool Vanilla by No7,NaN,No7,"956,696 in Beauty & Personal Care (","['B01B8BR0O8', 'B01B8BR0NO', 'B014MHXXM8']","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,28.76,7250468162,NaN,NaN,956696.0,Beauty & Personal Care,[No7 Stay Perfect Foundation now stays perfect...
3,wella koleston perfect hair colour medium int...,Wella Koleston Perfect Hair Colour 44/44 Mediu...,['B0041PBXX8'],NaN,"1,870,258 in Beauty & Personal Care (",NaN,"{'\n Item Weight: \n ': '1.76 ounces', '...",All Beauty,NaN,7367905066,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,1870258.0,Beauty & Personal Care,NaN
4,lacto calamine skin balance daily nourishing l...,Lacto Calamine Skin Balance Oil control 120 ml...,NaN,Pirmal Healthcare,"67,701 in Beauty & Personal Care (","['3254895630', 'B007VL1D9S', 'B00EH9A0RI', 'B0...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,12.15,7414204790,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,67701.0,Beauty & Personal Care,[Lacto Calamine Skin Balance Daily Nourishing ...


In [ ]:
# 檢查很多輪後決定把這些col拿掉
nouse = [ 'also_buy', 'brand', 'rank', 'also_view', 'details', 'main_cat', 'imageURL', 'imageURLHighRes', 'rank_c', 'description_n']
metadata = metadata.drop(nouse, axis=1)
metadata.head()


,description,title,price,asin,rank_n
0,loud n clear personal sound amplifier allows y...,Loud 'N Clear&trade; Personal Sound Amplifier,NaN,6546546450,2938573.0
1,lift luminate triple action serum by boots,No7 Lift &amp; Luminate Triple Action Serum 50...,44.99,7178680776,872854.0
2,stay perfect foundation now stays perfect for...,No7 Stay Perfect Foundation Cool Vanilla by No7,28.76,7250468162,956696.0
3,wella koleston perfect hair colour medium int...,Wella Koleston Perfect Hair Colour 44/44 Mediu...,NaN,7367905066,1870258.0
4,lacto calamine skin balance daily nourishing l...,Lacto Calamine Skin Balance Oil control 120 ml...,12.15,7414204790,67701.0


## nltk 

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
# Stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()


In [ ]:
def stem_text(text):
    """
    PorterStemmer is used for Stemming.
    """
    stemmer = PorterStemmer()
    return " ".join([stemmer.stem(word) for word in text.split()])


def rem_stopwords(text, stop_word="english"):
    eng_stop_words = stopwords.words(stop_word)
    return " ".join([word for word in text.split() if word not in eng_stop_words])


In [ ]:
metadata['description'] = metadata['description'].apply(rem_stopwords)
metadata['description'] = metadata['description'].apply(stem_text)

In [ ]:
metadata

,description,title,price,asin,rank_n
0,loud n clear person sound amplifi allow turn v...,Loud 'N Clear&trade; Personal Sound Amplifier,NaN,6546546450,2938573.0
1,lift lumin tripl action serum boot,No7 Lift &amp; Luminate Triple Action Serum 50...,44.99,7178680776,872854.0
2,stay perfect foundat stay perfect longer hour ...,No7 Stay Perfect Foundation Cool Vanilla by No7,28.76,7250468162,956696.0
3,wella koleston perfect hair colour medium inte...,Wella Koleston Perfect Hair Colour 44/44 Mediu...,NaN,7367905066,1870258.0
4,lacto calamin skin balanc daili nourish lotion...,Lacto Calamine Skin Balance Oil control 120 ml...,12.15,7414204790,67701.0
...,...,...,...,...,...
32887,bariel pro textur grip cuticl nipper purpl,"Barielle Pro Textured Grip Cuticle Nipper, Purple",9.95,B01HIWLLUK,2145325.0
32888,buy get free salon perfect eye makeup correcto...,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,NaN,B01HJ1K3YK,1639713.0
32889,dmannos mg veg capsul pack,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,55.63,B01HJ84SGM,207410.0
32890,brand new high qualiti enabl fast volley ensur...,12 White Feather Shuttlecocks Birdies Badminto...,12.99,B01HJASD20,965673.0


## 資料切分

In [ ]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [ ]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25


## 1. 助教示範

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 計算商品用標題所表示的 tfidf 矩陣
df = metadata.drop_duplicates('title')
tf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tf.fit_transform(df['title'])

# 計算商品間的相似程度
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['title'])

# 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品產生推薦
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res

In [ ]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    content-based
    '''
    ratings_trainings = training_data
    recommendations = {user: recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k) for user in users}
    return recommendations

ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

{'A100XQFWKQ30O2': [],
 'A103T1QOGFCSEH': [],
 'A106UKKSJ2KXPF': [],
 'A10A7GV4D5A11V': [],
 'A1119JJ37ZLB8R': [],
 'A113UOOLBSZN52': [],
 'A12M4U7WK4ALCR': [],
 'A12T8YTW6VWT7S': [],
 'A1364JXGKB46MM': [],
 'A137DALOQFKBTI': [],
 'A13FEZ3WV7S2EY': [],
 'A13IV4I1B0RXMG': [],
 'A13JU88JAHN72I': ['B00BQLFYUI',
  'B01BXFIBY2',
  'B0087IIY8M',
  'B00BMIWN5E',
  'B00EKSGOU8',
  'B01FKSC52W',
  'B00YHET2OA',
  'B00JYJ298O',
  'B00LG64HUG',
  'B008O60G8S'],
 'A13K55R6VH1OOD': [],
 'A13P7VFU075A': [],
 'A13SWYE4QLB6NG': [],
 'A13ZTQ0Q4ATA41': [],
 'A142EDN04OD62U': [],
 'A142I22FIC8MZK': [],
 'A14834QTII5TLT': [],
 'A14A447VPACTBC': [],
 'A14AP6MN5XO6LB': [],
 'A14CLF25IX25US': [],
 'A14LYXC3HTBAHI': [],
 'A14VUW4KZ34EOE': [],
 'A14Y32P26G9YL': [],
 'A157T25PBS7MX4': [],
 'A15HZDSERD85C8': [],
 'A15JJ8J1FGADIX': [],
 'A15ZCL70JXXH89': [],
 'A1617KN2IAWZ6J': [],
 'A16E0O88262HKA': [],
 'A16NSZ58PTVIYF': [],
 'A16UGDXRTDLJG5': [],
 'A16X9HR3UFQQXY': [],
 'A16Y7V1CZCWKFV': ['B017VQRMD8',
  'B00K6

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.0

In [ ]:
rec_by_title = pd.DataFrame.from_dict(ratings_by_user, orient='index')
rec_by_title.sort_values(by=10)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
ASGIVOW34XNQS,B000NT6AG6,B0184GV8KC,B009FLQD7E,B00QUHMTWE,B00IKU9FXA,B0009MW430,B00T4EL5WU,B018IWXF44,B001OIOJ0Y,B018T9QJYO,...,None,None,None,None,None,None,None,None,None,None
A1SA3N793WT2LM,B00757DZ6C,B002ZP7S12,B00WZVCOPG,B00JFBT916,B008UKB4F2,B00RXVR21E,B01EAG8312,B00103KS4E,B00H78RQMI,B002KPQ3RC,...,B015G8MDBE,B00GUXIZGW,B004Y46MK0,B000K8WZ7S,B001DB6AWM,B017MEMRS4,B00T0SXWLM,B00QZPCP0C,B00ERJZUE6,B00RYH7AI2
ACE1ZIOI3U6PY,B00AHCDFS0,B01095Q2MU,B00BEGEQNQ,B010DP05D8,B0053OX78U,B00SL2NPYM,B00NX5MPUM,B00782LTGW,B00BEGEQKO,B01D0SD00U,...,None,None,None,None,None,None,None,None,None,None
A4IQXCCXDEAKW,B001RYOR70,B00FQR9OEK,B000HA99LE,B005FYK6N2,B003HBDC6I,B01A4I1TR0,B009GO9UUC,B00KVMRNQQ,B000XEVN9A,B00T72SSTW,...,None,None,None,None,None,None,None,None,None,None
A3H8Z7JS7XJHJJ,B00TSRU754,B00UC8XP98,B004BPUY3S,B0015YL7YS,B00WQALW9U,B00Q8N2QHI,B01F756QQ4,B00UXE6OG2,B002L06STY,B000F6R6B0,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZECTAHZICNW5,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
AZEGZXCFTLSQB,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
AZEKC7DEXXJD7,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
AZFGSCB0S59LQ,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


觀察範例使用 title 去做推薦的output，結果會發現很多user並沒有接收到推薦商品。 然後有些人得到超多推薦，最多有到40個的。

想到上周有同學說，在testing dataset 裡的userid 有些過去並沒有買過，所以無法以他們的過去資料進行推薦。

In [ ]:
# 新舊客戶比例檢查
a = ratings_trainings['reviewerID'].unique()
a = pd.Series(a)
b = ratings_testings['reviewerID'].unique()
b = pd.Series(b)
c = a.isin(b)
print(c.sum()) # 只有38筆是舊客戶
print(38/len(b)) # 只佔testing 的客戶人數0.065, 所以如果直接使用客戶先前買過的商品去推薦的效果可能非常不好。

38
0.06506849315068493


In [ ]:
# 所以分成新舊兩組客戶做處理 
a=a.tolist()
ratings_testings_old = ratings_testings[ratings_testings['reviewerID'].map(lambda x:x in a )]
ratings_testings_new = ratings_testings[ratings_testings['reviewerID'].map(lambda x:x not in a )]
print(len(ratings_testings_old))

print(len(ratings_testings_new))
# 存成不同名單去處理 
old_list = ratings_testings_old['reviewerID'].tolist()
new_list = ratings_testings_new['reviewerID'].tolist()

39
551


## 先處理舊客戶

In [ ]:
df_old = ratings_testings[ratings_testings['reviewerID'].isin(old_list)]
# 找出舊客戶買過的東西
out_bought = df_old[['reviewerID', 'asin']]
out_bought = out_bought.sort_values(by='reviewerID')
out_bought.head()

,reviewerID,asin
343283,A13JU88JAHN72I,B017I6B6GK
362219,A16Y7V1CZCWKFV,B01E54U1RI
357765,A19503XX7GU6J2,B01CTQC9J2
362220,A1EY751AUVO5NZ,B01E54U1RI
357462,A1PR178735XC70,B01CRSWZAU


In [ ]:
# 計算商品用description_n所表示的 tfidf 矩陣
description_df = metadata['description']
tf = TfidfVectorizer(min_df=2, max_df=0.7, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(description_df)
# print(tf.get_feature_names())
# print(len(tf.get_feature_names()))

In [ ]:
# 計算商品間的相似程度
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(description_df.index, index = metadata['asin'])

In [ ]:
similarity_matrix.shape

(31712, 31712)

In [ ]:
cosine_similarity_df = pd.DataFrame(similarity_matrix, 
                                    index=mapping.index, 
                                    columns=mapping.index)

# asin 之間的相似性參照表 
cosine_similarity_df.head()

asin,6546546450,7178680776,7250468162,7367905066,7414204790,7535842801,7538626107,8279996397,8637910351,8867326759,...,B01HIM4GZ2,B01HIM3A3G,B01HIPOQ2M,B01HIUEEHO,B01HIWKGOM,B01HIWLLUK,B01HJ1K3YK,B01HJ84SGM,B01HJASD20,B01HJEGTYK
asin,,,,,,,,,,,,,,,,,,,,,
6546546450,1.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
7178680776,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
7250468162,0.0,0.0,1.000000,0.090848,0.029533,0.0,0.021761,0.0,0.0,0.037485,...,0.021894,0.018513,0.0,0.0,0.0,0.0,0.057802,0.000000,0.015467,0.000000
7367905066,0.0,0.0,0.090848,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.057529,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.033058,0.000000,0.000000,0.072938
7414204790,0.0,0.0,0.029533,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.034463,0.0,0.0,0.0,0.0,0.027590,0.050344,0.000000,0.000000


In [ ]:
# 每個商品回傳 k 個最相近的商品
# item_input 就是客戶買過的東西
# mapping就是那個asin
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (metadata['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品產生推薦
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res

In [ ]:
# 舊客戶買的東西來做推薦清單
def recommender_old(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    content-based
    '''
    ratings_trainings = training_data
    recommendations = {user: recommend_items(out_bought[out_bought['asin'].isin(out_bought[out_bought['reviewerID'] == user]['asin'].tolist())]['asin'].tolist(), k) for user in users}
    return recommendations

ratings_by_old_user = recommender_old(out_bought, users)
ratings_by_old_user


{'A100XQFWKQ30O2': [],
 'A103T1QOGFCSEH': [],
 'A106UKKSJ2KXPF': [],
 'A10A7GV4D5A11V': [],
 'A1119JJ37ZLB8R': [],
 'A113UOOLBSZN52': [],
 'A12M4U7WK4ALCR': [],
 'A12T8YTW6VWT7S': [],
 'A1364JXGKB46MM': [],
 'A137DALOQFKBTI': [],
 'A13FEZ3WV7S2EY': [],
 'A13IV4I1B0RXMG': [],
 'A13JU88JAHN72I': ['B019213AE0',
  'B0010MGGSC',
  'B00ONSGKBS',
  'B007AZ3CEO',
  'B006Z7TE5E',
  'B01CWFI5TI',
  'B00RKFXEOM',
  'B00N1BURPS',
  'B019DD2PRU',
  'B00YNSV8T2'],
 'A13K55R6VH1OOD': [],
 'A13P7VFU075A': [],
 'A13SWYE4QLB6NG': [],
 'A13ZTQ0Q4ATA41': [],
 'A142EDN04OD62U': [],
 'A142I22FIC8MZK': [],
 'A14834QTII5TLT': [],
 'A14A447VPACTBC': [],
 'A14AP6MN5XO6LB': [],
 'A14CLF25IX25US': [],
 'A14LYXC3HTBAHI': [],
 'A14VUW4KZ34EOE': [],
 'A14Y32P26G9YL': [],
 'A157T25PBS7MX4': [],
 'A15HZDSERD85C8': [],
 'A15JJ8J1FGADIX': [],
 'A15ZCL70JXXH89': [],
 'A1617KN2IAWZ6J': [],
 'A16E0O88262HKA': [],
 'A16NSZ58PTVIYF': [],
 'A16UGDXRTDLJG5': [],
 'A16X9HR3UFQQXY': [],
 'A16Y7V1CZCWKFV': ['B01FTG4VMM',
  'B004I

In [ ]:
evaluate(ratings_by_old_user, ratings_by_user)

0.0

## 新客戶用 rule-based

1. 檢查每個美妝產品的第一次被評論日期和最後一次被評論日期


In [ ]:
review_date = ratings_trainings.groupby('asin', as_index=False).agg(review_1st = ('DATE', 'min'), review_last = ('DATE', 'max'))


In [ ]:
display(review_date.sort_values('review_last'))

,asin,review_1st,review_last
0,0061073717,2000-05-06,2000-06-03
115,B000050B67,2001-01-02,2001-09-03
114,B000050B65,2001-01-15,2002-08-19
191,B00006FRW1,2002-11-03,2002-11-03
111,B000050B62,2000-10-29,2003-02-20
...,...,...,...
29515,B01DJI7796,2016-05-15,2018-08-31
28904,B01CW8SDLU,2017-11-20,2018-08-31
28903,B01CW24JXC,2016-08-06,2018-08-31
28429,B01CD7JK9E,2016-08-13,2018-08-31


In [ ]:
review_date['review_last']=pd.to_datetime(review_date['review_last'])
review_date['review_1st']=pd.to_datetime(review_date['review_1st'])


review_date['review_last_year']=review_date['review_last'].dt.year
review_date['review_1st_year']=review_date['review_1st'].dt.year

In [ ]:
display(review_date.sort_values('review_last'))

review_date['review_last_year'].value_counts().sort_values()


,asin,review_1st,review_last,review_last_year,review_1st_year
0,0061073717,2000-05-06,2000-06-03,2000,2000
115,B000050B67,2001-01-02,2001-09-03,2001,2001
114,B000050B65,2001-01-15,2002-08-19,2002,2001
191,B00006FRW1,2002-11-03,2002-11-03,2002,2002
111,B000050B62,2000-10-29,2003-02-20,2003,2000
...,...,...,...,...,...
29515,B01DJI7796,2016-05-15,2018-08-31,2018,2016
28904,B01CW8SDLU,2017-11-20,2018-08-31,2018,2017
28903,B01CW24JXC,2016-08-06,2018-08-31,2018,2016
28429,B01CD7JK9E,2016-08-13,2018-08-31,2018,2016


2000       1
2001       1
2002       2
2003       6
2004      17
2006      29
2005      39
2007      90
2008     138
2009     248
2010     282
2011     348
2012     417
2013    1085
2014    2163
2015    4575
2018    6580
2017    8018
2016    8543
Name: review_last_year, dtype: int64

可以看出有一些產品已經很久沒有人評論，可能是已經不在市面上，所以僅保留近5年的做推薦試看看 (2014之後)

In [ ]:
import datetime 
ratings_trainings['DATE']=pd.to_datetime(ratings['DATE'])
ratings_trainings['year']=ratings_trainings['DATE'].dt.year
ratings_trainings['month']=ratings_trainings['DATE'].dt.month
ratings_trainings['quarter']=ratings_trainings['DATE'].dt.quarter

In [ ]:
## 只保留近5年，Q3的販售商品, 使用這個去套用上次的rule base
ratings_trainings_Q3 = ratings_trainings[(ratings_trainings['year']>2013)& (ratings_trainings['quarter']==3)]
print(len(ratings_trainings_Q3)) 
print(len(ratings_trainings))

77995
370752


In [ ]:
# 讓rule_base 僅用於新客戶，所以先做一個新客戶的清單
new_list = pd.Series(new_list)
users = pd.Series(users)
users_new = users[users.isin(new_list)]
print(len(new_list))
print(len(users))
print(len(users_new))

551
584
546


In [ ]:
def recommender_2(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    popularity = training_data["asin"].value_counts().index.to_list()[0:k]
    recommendations = {x: popularity for x in users}
    return recommendations

ratings_by_new_user = recommender_2(ratings_trainings_Q3, users_new)

In [ ]:
evaluate(ratings_by_new_user, ratings_by_user)

0.0

In [ ]:
# 把新舊名單與推薦商品合併
ratings_testings_by_user = {}
for d in [ratings_by_old_user, ratings_by_new_user]:
  ratings_testings_by_user.update(d)

In [ ]:
evaluate(ratings_testings_by_user, ratings_by_user)

0.0

In [ ]:
ratings_testings_by_user

{'A100XQFWKQ30O2': ['1620213982',
  'B01DKQAXC0',
  'B00W259T7G',
  'B001QY8QXM',
  'B000FOI48G',
  'B000GLRREU',
  'B006IB5T4W',
  'B00VF344X0',
  'B0010ZBORW',
  'B0067F28ZW'],
 'A103T1QOGFCSEH': ['1620213982',
  'B01DKQAXC0',
  'B00W259T7G',
  'B001QY8QXM',
  'B000FOI48G',
  'B000GLRREU',
  'B006IB5T4W',
  'B00VF344X0',
  'B0010ZBORW',
  'B0067F28ZW'],
 'A106UKKSJ2KXPF': ['1620213982',
  'B01DKQAXC0',
  'B00W259T7G',
  'B001QY8QXM',
  'B000FOI48G',
  'B000GLRREU',
  'B006IB5T4W',
  'B00VF344X0',
  'B0010ZBORW',
  'B0067F28ZW'],
 'A10A7GV4D5A11V': ['1620213982',
  'B01DKQAXC0',
  'B00W259T7G',
  'B001QY8QXM',
  'B000FOI48G',
  'B000GLRREU',
  'B006IB5T4W',
  'B00VF344X0',
  'B0010ZBORW',
  'B0067F28ZW'],
 'A1119JJ37ZLB8R': ['1620213982',
  'B01DKQAXC0',
  'B00W259T7G',
  'B001QY8QXM',
  'B000FOI48G',
  'B000GLRREU',
  'B006IB5T4W',
  'B00VF344X0',
  'B0010ZBORW',
  'B0067F28ZW'],
 'A113UOOLBSZN52': ['1620213982',
  'B01DKQAXC0',
  'B00W259T7G',
  'B001QY8QXM',
  'B000FOI48G',
  'B000

## QQ
曾經在自己測試的時候有跑出過0.55的版本，但不知道為什麼在整理code的時候不見了, 之後怎麼跑都是0.0的結果